![Banner](images/dsep-banner.png)
# **Welcome to the Applied Methods for Social Sciences in Python Workshop**

By John, Barry, and Simran

In Collaboration with the Division of Data Science's [Data Peer Consulting](https://data.berkeley.edu/ds-peer-consulting)

### BEFORE WE BEGIN PLEASE COMPLETE THIS SURVEY
[Pre-Workshop Survey Link](https://forms.gle/e1Dq8NcJF8xPye6N8)  

## John Park
![John](https://data.berkeley.edu/sites/default/files/styles/width_400/public/john_pic_-_john_park_0.jpg?itok=-kg9pNQg&timestamp=1599267808)

Quick Facts About Me:

    🐻 Senior at Cal
    🎒 Studying Computer Science and Economics
    🏢 Interned and returning full-time as an SDE@Amazon
    📊 Joined the Data Peer Consulting team in Fall 2018

How to Reach Me:

    📮 Email: jhp@berkeley.edu

## Bharadwaj Swaminathan

![Barry](images/barry.jpg)

Quick Facts About Me:

    🐻 Senior at Cal
    🎒 Studying Data Science and Economics
    🏢 COVID-19 researcher at University of West Florida
    📊 Joined the Data Peer Consulting team in Fall 2018

How to Reach Me:

    📮 Email: bharadwaj.s@berkeley.edu

## Simran Sachdev
![Simran](https://data.berkeley.edu/sites/default/files/styles/width_400/public/headshot_-_simran_sachdev.jpg?itok=PgdDBm5M&timestamp=1599267430)

Quick Facts About Me:

    🐻 Senior at Cal
    🎒 Studying Data Science and Applied Math
    🏢 Data Scientist Intern at Boston Scientific
    📊 Joined the Data Peer Consulting team in Spring 2020

How to Reach Me:

    📮 Email: ssach@berkeley.edu

<a class="anchor" id="tof"></a>
## Table of Contents

Use anchors to set these hyperlinks to jump to certain locations in the notebook. 

- [Introduction](#1)
- [Example](#2)
- [Reference Sheets](#rs)

---

## Workshop Goals

The goal of this workshop is to cover the fundamental tools offered in Python for applied methods in the social sciences. 

    - Learn how to import and modify data in Python via Pandas
    - Apply OLS and related statistical techniques
    - Demonstrate a basic example workflow from raw data to analysis
    
Specifically, we will be working with `statsmodels`. This package provides a wide range of useful statistical tools for the social sciences, including but not limited to least squares, panel methods, mixed models, etc.

--- 

## Why Python?

Python is one of the most popular general-purpose computing languages, and for good reason. Python's readability, maintainability, and robust community support, especially in the IPython sphere, makes it a strong choice for data science, including for the social sciences. 

### Transitioning from R

`statsmodels` provides support for R-like formula expressions. You can read more about it [here](https://www.statsmodels.org/stable/examples/notebooks/generated/formulas.html).

<a class="anchor" id="1"></a>

## Introduction 

Lets go through a basic example from the [statsmodels documentation](https://www.statsmodels.org/stable/gettingstarted.html). The dataset is a collection of historical data used in support of Andre-Michel Guerry’s 1833 Essay on the Moral Statistics of France.

In [ ]:
### standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [ ]:
# Grab sample data from Statsmodels dataset repo

df = sm.datasets.get_rdataset("Guerry", "HistData").data

In [ ]:
### Note that the data is already in a Pandas DataFrame

type(df)

In [ ]:
df

Lets explore and see if literacy rates are associated with per capita entries in the Royal Lottery. Our model will also need to control for wealth in each 'department'.

We will use OLS to estimate this model, as described below:

$$y = {\beta}X + \epsilon$$

Where $y$ is a $N * 1$ column of per-capita lottery wagers (Lottery), while $X$ is an $N * 3$ matrix with a constant, literacy, and wealth.

We know then that the OLS estimates on the coefficients will be:

$$\hat{\beta} = (X'X)^{-1}X'y$$

To actually implement a model, there are a couple options, especially with regards to how we could encode dummy variables (if we had them). For this example, we'll be using Patsy.

## Approach Using Patsy


Patsy provides a convenient way to encode design matrcies using R-like formulas. You can see the docs [here](https://patsy.readthedocs.io/en/latest/). 


In [ ]:
from patsy import dmatrices

In [ ]:
y, X = dmatrices('Lottery ~ Literacy + Wealth', data=df, return_type='dataframe')

Here's what the resulting design matrix/data frame looks like

In [ ]:
y.head()

In [ ]:
X.head()

Note that the resultant matrix/data

- includes a constant to the exog regressors matrix (very useful)
- are pandas dataframes

Let's now fit the model! The workflow for statsmodels OLS is straightforward

- 1. Define X, y (we just did this)
- 2. Choose the appropriate statsmodels model class, and define a model (we'll use sm.OLS)
- 3. Fit the model and inspect results

In [ ]:
### Define model

model1 = sm.OLS(y, X)

### Fit model

results1 = model1.fit()

### Inspect model

print(results1.summary())

We can also extract raw result data via the results object attributes. You can read more [here](https://www.statsmodels.org/stable/regression.html).

In [ ]:
### all extractable attributes 

print(dir(results1))

In [ ]:
### grab coefficients

results1.params

In [ ]:
### Or get rsquared

results1.rsquared

## Testing your model (basic edition)

Statsmodels gives us a [range](https://www.statsmodels.org/stable/stats.html#residual-diagnostics-and-specification-tests) of diagnostic and specification tests. 

Let's try out the Rainbow test of linearity: the null is that the model is correct in being linear

In [ ]:
sm.stats.linear_rainbow(results1)

The first value we will get from the test is the F-stat while the second is the p-value. As our p-value is comfortably above our rejection region (standard is $\alpha = 0.05$), we know that modelling this problem as linear is not incorrect.

## Your turn! (10 min)

Let's try running this regression, but now controlling for regional heterogeniety. In other words, **include** *region* by creating dummy variables representing each region. $X$ should now be a $N * 7$.

**Note:** Patsy handles this for you ;)

In [ ]:
### What should our fomula be?
y, X = dmatrices('...', data=df, return_type='dataframe')

In [ ]:
### Define your model

model = ...

### Fit your model

results = ...

### Inspect your model

print(...)

In [ ]:
### Test your model

print('Rainbow test pval: ' + str(sm.stats.linear_rainbow(results)[1]))100jjkjjj

How do our results here compare to the model in which we controlled for region? Take a look at the rsquared, as well as the error bars on the coefficients. Discuss this within your group :)

<a class="anchor" id="2"></a>

##         Example: Contingent valuation

The file <code>LoomistForestCVDataset.csv</code> contains a subset of the dataset used by Loomis et al. (1996). The dataset consists of five columns. The first column lists a bid amount randomly proposed to a respondent to assess their willingness to pay for a fire management program for old growth Pacific Northwest forests. The second column lists the number of respondents saying yes to the project under version one of the survey. The third column lists the number of respondents saying no to the project under version one of the survey. Columns four and five give yes and no responses for the same bid amount but under a different version of the survey. 


Credit to Prof. Graham's Ec141 course materials for this excercise. Check out his github [here](https://github.com/bryangraham).

In [ ]:
loomisCVD = pd.read_csv("./data/LoomisCVData.csv")
loomisCVD

1. Lets begin by writing a short Python script to transform the given datafile into one respondent per row form. The first column should equal $D = 1$ if the respondent answered yes, $D = 0$ if they answered no. The second column should give the bid amount, $A$. The third column is $X = 1$ if the response was solicited from version one of the survey and $X = 0$ if it was from version two. 

In [ ]:
def transform_data(df):
    D_ = []
    A_ = []
    X_ = []
    
    def add_trow(d, a, x):
        D_.append(d)
        A_.append(a)
        X_.append(x)

    for index, row in df.iterrows():
        b = row['BidAmount']
        for i in range(row['NumYes_v1']):
            add_trow(1,b,1)
        for i in range(row['NumNo_v1']):
            add_trow(0,b,1)
        for i in range(row['NumYes_v2']):
            add_trow(1,b,0)
        for i in range(row['NumNo_v2']):
            add_trow(0,b,0)
            
    return pd.DataFrame(columns=['D', 'A', 'X'], data=np.array([D_, A_, X_]).T)

In [ ]:
cvd_transformed = transform_data(loomisCVD)

### transformed data set
print(cvd_transformed)

2. Let's assume that willingness-to-pay for the fire management program for a randomly sampled person is: $$ W = \alpha + X'\beta + V$$ where $V | X, A ∼ \it  \mathcal{N}(0, \sigma^2)$ captures heterogeneity in willingness-to-pay across individuals. 

(Is this assumption robust? See errata)

3. Assume that individuals respond yes to the proposal if their willingness-to-pay exceeds the bid they were offered. Then:

$$Pr (D = 1| X, A) = \Phi\left(\frac \alpha \sigma - \frac 1 \sigma A + X' \frac \beta \sigma \right)$$ with $\Phi(*)$ the CDF of the standard normal distribution.

(See errata for proof)

4. Let's use probit regression analysis to construct estimates of the composite parameters $\frac \alpha \sigma$, $- \frac 1 \sigma$, $\frac \beta \sigma$. From these estimates we'll recover estimates of the fundamental preference parameters $\alpha$, $\beta$ and $\sigma$. We'll implement a bootstrap procedure to construct standard error estimates for these parameters. 

Why/what does bootstrapping allow us to do? Why probit vs logit? 

In [ ]:
#define endog, explanatory variables
X = sm.add_constant(cvd_transformed[['A', 'X']])
y = cvd_transformed['D']

#probit model
probit_model = sm.Probit(y, X)
results = probit_model.fit()
param_estimates = results.params
print(param_estimates)

In [ ]:
### Bootstrap Procedure: Probit on const, A, X

B = 1000
N = 260

def param_bootstrap(df):
    const = []
    A_ = []
    X_ = []
    for i in range(B):
        
        sample_df = df.sample(n=N, replace=True)
        
        # probit procedure from earlier
        X = sm.add_constant(sample_df[['A', 'X']])
        y = sample_df['D']
        
        probit_model = sm.Probit(y, X)
        results = probit_model.fit()
        
        const.append(results.params['const'])
        A_.append(results.params['A'])
        X_.append(results.params['X'])
    return pd.DataFrame(columns=['alpha', 'sigma', 'beta'], data=np.array([const, A_, X_]).T)

In [ ]:
bootstrap_param_results = param_bootstrap(cvd_transformed)

In [ ]:
#estimated parameters
estimate_sigma = -(1 / param_estimates['A'])
estimate_beta = estimate_sigma * param_estimates['X']
estimate_alpha = estimate_sigma * param_estimates['const']

#estimated parameters from boot strap
sigma_bs = -(1/bootstrap_param_results['sigma'])
beta_bs = bootstrap_param_results['beta'] * sigma_bs
alpha_bs = sigma_bs * bootstrap_param_results['alpha']

bs_params=pd.DataFrame(columns=['alpha', 'sigma', 'beta'], data=np.array([alpha_bs, sigma_bs, beta_bs]).T)

print(estimate_alpha,estimate_sigma, estimate_beta)
print(bs_params.describe())

Our recovered parameters from our probit regression onto the dataset is:

$$\hat{\alpha} = 89.93$$
$$\hat{\sigma} = 104.96$$
$$\hat{\beta} = -16.86$$

Our bootstrap recovered params have the characteristics as shown in the table above, with recovered standard errors as follows:

$$SE(\hat{\alpha})  = 13.22 $$
$$SE(\hat{\sigma}) = 17.44 $$
$$SE(\hat{\beta}) = 17.80 $$

### Discussion:

Our results for the parameters were as expected; the sign of the coefficient on beta is interesting however. Loomis, in the literature, found no statistical difference between the two surveys. Indeed, although our computed coefficient on beta is less than zero, it falls well within a $0.05$-significance around zero (which indicates that we also found no statistical difference between the surveys, as our confidence band would be $[-35.6, 35.6]$, which our computed beta falls within). Thus our results fall in line with what was expected both from the literature and intuitively (such as higher bid prices indicating lower probability of support). 

You are part of an environmental conservation group that is campaigning for a ballot initiative that would fund a fire management program like the one studied by Loomis et al. (1996). The type of initiative you wrote needs to pass with a majority of 67 percent. Your organization wrote the ballot initiative with a proposed tax of $\hat{A}^{∗} −0.05$ per person, with $\hat{A}^{∗}$ equal to $$ \hat{A}^{∗} = \hat{\alpha} − \hat{\sigma}\Phi^{-1}(0.67)$$ Here $\hat{\alpha}$ and $\hat{\sigma}$ correspond to your point estimates from question 4 above. Explain the
reasoning behind choosing the proposed tax in this way? Construct an estimate of this
tax (as well as a standard error using the bootstrap).

In [ ]:
from scipy.stats import norm

tax_estimate = estimate_alpha - estimate_sigma*norm.ppf(0.67) - 0.05
print(tax_estimate)

In [ ]:
### Bootstrap Procedure: Tax = A^* - 0.05*

B = 10000
N = 260
def a_est_bootstrap(df):
    
    a_hats = []
    
    for i in range(B):
        
        #sample with replacement
        sample_df = df.sample(n=N, replace=True)
        
        X = sm.add_constant(sample_df[['A', 'X']])
        y = sample_df['D']
        
        probit_model = sm.Probit(y, X)
        results = probit_model.fit()
        
        sigma_hat = -(1 / results.params['A'])
        alpha_hat = sigma_hat * results.params['const']
        
        a_hat_estimate = alpha_hat - (sigma_hat*norm.ppf(0.67)) - 0.05
        a_hats.append(a_hat_estimate)
        
    return pd.DataFrame(columns=['a_hat'], data=np.array([a_hats]).T)

In [ ]:
a_hat_bs = a_est_bootstrap(cvd_transformed)
a_hat_bs

In [ ]:
print(a_hat_bs.describe())

We construct the tax in this fashion in order to target the WTP of at least $67\%$ of the population (i.e. to find the tax-level that is less than or equal to the WTP of at least $67\%$ of the population). This is ensured by the $\hat{\sigma}\Phi^{-1}(0.67)$ term, as it basically moves $\hat{\alpha}$ (the mean willingness to pay) by the amount needed to capture $67\%$ of the sampled population willingness to pay. 

We find our estimate of the tax to be: $43.70$ and the standard error via bootstrap to be $12.81$ as shown in the result above.

<a class="anchor" id="rs"></a>
### Reference Sheets!
[Back to Table of Contents](#tof)

Links updated as of 1/1/11.

- [NumPy Cheat Sheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Numpy_Python_Cheat_Sheet.pdf)  
- [Pandas Cheat Sheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)  
- [Matplotlib Cheat Sheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Python_Matplotlib_Cheat_Sheet.pdf)  
- [Seaborn Cheat Sheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Python_Seaborn_Cheat_Sheet.pdf)

Documentation

- [Statsmodels Docs](https://www.statsmodels.org/stable/index.html)
- [Linearmodels Docs](https://bashtage.github.io/linearmodels/)


### More Resources
1. Data Peer Consultants - That's us! We help undergrads and graduate students with projects, research, and more! Come to our drop-in hours.  
https://data.berkeley.edu/ds-peer-consulting

2. Towards Data Science - Website full of good blogs and helpful introductions to data science stuff.  
https://towardsdatascience.com/

3. Stack Overflow // Google - A great data scientist is adept at using StackOverflow and Google to find the answers to their bugs. More likely than not, someone out there has ran into the exact same problem as you, so might as well use their solutions as a resource!

## Thanks for Coming! PLEASE COMPLETE THIS POST-WORKSHOP SURVEY!  
[Post-Workshop Survey](https://forms.gle/A7Dt5pADe57XdWBt8)

### Errata

#### 2. 
The survey design ensures independence of $V$ and $X$ and $A$ as $X$ and $A$ are randomly assigned to the population without any selection critereon. More specifically, the general random selection of surveyees coupled with the further random assignment of $X$ (via survey version) ensures independence by construction. 

#### 3.

We begin our analysis by examining the point of indifference between $W$ and $A$:

$$A = W$$

Standardize

$$\frac A \sigma = \frac W \sigma$$

Expand $W$

$$\frac{A}{\sigma} = \frac{\alpha}{\sigma} + X^{'} \frac{\beta}{\sigma} + \frac{V}{\sigma}$$

Normalize to mean zero:

$$\frac{\alpha}{\sigma} - \frac{1}{\sigma} A + X^{'} \frac{\beta}{\sigma} + \frac{V}{\sigma}$$

Let the above expression be $Y$. We realize that:

$$Pr(D=1|X,A) = Pr(Y > 0 | X, A)$$

Substituting:

$$Pr(D=1|X,A) = Pr\left(\frac{\alpha}{\sigma} - \frac{1}{\sigma} A + X^{'} \frac{\beta}{\sigma} + \frac{V}{\sigma} > 0 | X, A \right)$$


$$ = Pr\left(\frac{\alpha}{\sigma} - \frac{1}{\sigma} A + X^{'} \frac{\beta}{\sigma} + \frac{V}{\sigma}> 0 | X, A \right)$$


$$ = Pr\left(\frac{V}{\sigma} > -\frac{\alpha}{\sigma} + \frac{1}{\sigma} A - X^{'} \frac{\beta}{\sigma}| X, A \right)$$

By symmetry of the normal distribution and as $V | X, A ∼ \it  \mathcal{N}(0, \sigma^2)$:

$$ = Pr\left(\frac{V}{\sigma} < \frac{\alpha}{\sigma} - \frac{1}{\sigma} A + X^{'} \frac{\beta}{\sigma}| X, A \right)$$

Thus:

$$ Pr(D=1|X,A) = \Phi\left(\frac{\alpha}{\sigma} - \frac{1}{\sigma} A + X^{'} \frac{\beta}{\sigma}\right)$$